# GeoProcessing in Python

Student: Daniel Kuhlen

Instructors: 
- Dirk Pflugmacher
- Matthias Baumann

## Final Paper: NLD in El Salvador
This file downloads subnational population numbers for the central american countries.

In [14]:
import os
import ee
import folium
import pandas as pd
ee.Initialize()

# Visual Check for Population Density

In [35]:
### create a map for visual check ###

# define country boundary
el_salvador = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017') \
    .filter(ee.Filter.eq('country_co', 'ES'))

# access population density with gee
gpw_v411 = ee.Image('CIESIN/GPWv4/population-density/2020') \
    .clip(el_salvador)

# visualization parameters 
visualization = {
    'min': 0.0,
    'max': 400.0,
    'palette': ['ffffcc', 'ffeda0', 'fed976', 'feb24c', 'fd8d3c', 'fc4e2a', 'e31a1c', 'bd0026', '800026']
}

# add image to map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer
my_map = folium.Map(location=[13.794185, -88.89653], zoom_start=8)  # Coordinates roughly in the center of El Salvador

# add the population density layer to the map.
my_map.add_ee_layer(gpw_v411, visualization, 'Population Density')

# layer control panel to the map.
my_map.add_child(folium.LayerControl())

# plot the map
my_map


# Download Population Density on Subnational Level 1 Districts

In [37]:
### access the level 1 boundaries

# list of all countries
central_american_countries = ['Belize', 'Costa Rica',
                              'El Salvador', 'Guatemala',
                              'Honduras', 'Nicaragua',
                              'Panama']

# select level 1 admin boundaries for all countries
admin_boundaries = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level1') \
    .filter(ee.Filter.inList('ADM0_NAME', central_american_countries))

In [43]:
### access the gridded population density
gpw_v411 = ee.Image('CIESIN/GPWv4/population-density/2020')

# mean population density by region
density_by_admin = gpw_v411.reduceRegions(
    collection=admin_boundaries,
    reducer=ee.Reducer.mean(),
    scale=1000,
)

# extract results
extracted_data = density_by_admin.getInfo()

# create dataframe
data = {
    'Country': [feature['properties']['ADM0_NAME'] for feature in extracted_data['features']],
    'Admin Area': [feature['properties']['ADM1_NAME'] for feature in extracted_data['features']],
    'Population Density': [feature['properties']['mean'] for feature in extracted_data['features']],
}

# convert to pandas dataframe
pop_density = pd.DataFrame(data)

### export
filepath = '../data/output/population/pop_density.csv'

# save file
pop_density.to_csv(filepath,
          index=False)